In [3]:
from isochrones.dartmouth import Dartmouth_Isochrone
dar = Dartmouth_Isochrone()

3 stars physically bound wih mass 0.5Msun, 1.0Msun , 2.0Msun

In [19]:
from isochrones.utils import addmags
import numpy as np
M1 = 2.
M2 = 1.0
M3 = 0.5
age = np.log10(5e8)
feh = 0.0
distance = 500
AV = 0.1

unresolved_bands = ['i','J','H','K','W1','W2']
resolved_bands = ['i','J','K']
args = (age, feh, distance, AV)
unresolved = {b:addmags(dar.mag[b](M1, *args), dar.mag[b](M2, *args), dar.mag[b](M3, *args)) for b in unresolved_bands}
resolved_1 = {b:dar.mag[b](M1, *args) for b in resolved_bands}
resolved_2 = {b:dar.mag[b](M2, *args) for b in resolved_bands}
resolved_3 = {b:dar.mag[b](M3, *args) for b in resolved_bands}

In [20]:
unresolved, resolved_1, resolved_2, resolved_3

({'H': 9.7256583693171077,
  'J': 9.779566102660386,
  'K': 9.7007141247839961,
  'W1': 9.6944031577453735,
  'W2': 9.6931598667184637,
  'i': 10.250277342930602},
 {'J': 9.8842517232839757, 'K': 9.8482277980788435, 'i': 10.315409572448191},
 {'J': 12.457760436489632, 'K': 12.069009619730219, 'i': 13.37534244453469},
 {'J': 15.158660756096186, 'K': 14.324377065621592, 'i': 17.000775827153767})

In [23]:
import pandas as pd

instruments = ['twomass','WISE','SDSS','RAO']
bands = {'twomass':['J','H','K'],
          'WISE':['W1','W2'],
          'SDSS':['i'],
          'RAO':['i','J','K']}
mag_unc = {'twomass': 0.02, 'WISE':0.02, 'SDSS':0.02, 'RAO':0.1}
resolution = {'twomass':4.0, 'WISE':5.0, 'SDSS':4.5, 'RAO':0.1}
relative = {'twomass':False, 'WISE':False, 'SDSS':False, 'RAO':True}
separation2 = 0.5
separation3 = 1.2
PA2, PA3 = 100.,45.

columns = ['name', 'band', 'resolution', 'relative', 'separation', 'pa', 'mag', 'e_mag']
df = pd.DataFrame(columns=columns)
i=0
for inst in ['twomass', 'WISE','SDSS']:  #Unresolved observations
    for b in bands[inst]:
        row = {}
        row['name'] = inst
        row['band'] = b
        row['resolution'] = resolution[inst]
        row['relative'] = relative[inst]
        row['separation'] = 0.
        row['pa'] = 0.
        row['mag'] = unresolved[b]
        row['e_mag'] = mag_unc[inst]
        df = df.append(pd.DataFrame(row, index=[i]))
        i += 1

for inst in ['RAO']:  #Resolved observations
    for b in bands[inst]:
        mags = [resolved_1[b], resolved_2[b], resolved_3[b]]
        pas = [0, PA2, PA3]
        seps = [0., separation2, separation3]
        for mag,sep,pa in zip(mags,seps,pas):
            row = {}
            row['name'] = inst
            row['band'] = b
            row['resolution'] = resolution[inst]
            row['relative'] = relative[inst]
            row['separation'] = sep
            row['pa'] = pa
            row['mag'] = mag
            row['e_mag'] = mag_unc[inst]
            df = df.append(pd.DataFrame(row, index=[i]))
            i += 1
            
df

,band,e_mag,mag,name,pa,relative,resolution,separation
0,J,0.02,9.779566,twomass,0.0,False,4.0,0.0
1,H,0.02,9.725658,twomass,0.0,False,4.0,0.0
2,K,0.02,9.700714,twomass,0.0,False,4.0,0.0
3,W1,0.02,9.694403,WISE,0.0,False,5.0,0.0
4,W2,0.02,9.693160,WISE,0.0,False,5.0,0.0
5,i,0.02,10.250277,SDSS,0.0,False,4.5,0.0
6,i,0.10,10.315410,RAO,0.0,True,0.1,0.0
7,i,0.10,13.375342,RAO,100.0,True,0.1,0.5
8,i,0.10,17.000776,RAO,45.0,True,0.1,1.2
9,J,0.10,9.884252,RAO,0.0,True,0.1,0.0


In [25]:
from isochrones.observation import ObservationTree

In [28]:
t = ObservationTree.from_df(df, name='test-triplet')
t.print_ascii()

test-triplet
 ╚═ WISE W1=(9.69, 0.02) @(0.00, 0 [5.00])
    ╚═ WISE W2=(9.69, 0.02) @(0.00, 0 [5.00])
       ╚═ SDSS i=(10.25, 0.02) @(0.00, 0 [4.50])
          ╚═ twomass H=(9.73, 0.02) @(0.00, 0 [4.00])
             ╚═ twomass J=(9.78, 0.02) @(0.00, 0 [4.00])
                ╚═ twomass K=(9.70, 0.02) @(0.00, 0 [4.00])
                   ╠═ RAO J=(9.88, 0.10) @(0.00, 0 [0.10])
                   ║  ╚═ RAO K=(9.85, 0.10) @(0.00, 0 [0.10])
                   ║     ╚═ RAO i=(10.32, 0.10) @(0.00, 0 [0.10])
                   ╠═ RAO J=(12.46, 0.10) @(0.50, 100 [0.10])
                   ║  ╚═ RAO K=(12.07, 0.10) @(0.50, 100 [0.10])
                   ║     ╚═ RAO i=(13.38, 0.10) @(0.50, 100 [0.10])
                   ╚═ RAO J=(15.16, 0.10) @(1.20, 45 [0.10])
                      ╚═ RAO K=(14.32, 0.10) @(1.20, 45 [0.10])
                         ╚═ RAO i=(17.00, 0.10) @(1.20, 45 [0.10])
